In [ ]:
import os
from flask import Flask, request, render_template, redirect, url_for
import joblib
import numpy as np
from PIL import UnidentifiedImageError
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'static/uploads'
app.config['STATIC_FOLDER'] = 'static'

# Load the pre-trained VGG16 model + higher level layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Load the SVM model
model_filename = 'DC.joblib'
try:
    svm = joblib.load(model_filename)
    print(f'SVM model loaded from {model_filename}')
except FileNotFoundError:
    print(f"Error: The model file '{model_filename}' was not found.")
    exit()
except Exception as e:
    print(f"Error loading the model: {e}")
    exit()

def preprocess_image(image_path):
    """Preprocesses the image using VGG16 preprocessing for SVM prediction."""
    try:
        img = load_img(image_path, target_size=(150, 150))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        features = base_model.predict(img_array)
        features = np.reshape(features, (features.shape[0], -1))  # Flatten the features
        return features
    except FileNotFoundError:
        print(f"Error: The file '{image_path}' was not found.")
    except UnidentifiedImageError:
        print(f"Error: The file '{image_path}' is not a valid image.")
    except Exception as e:
        print(f"Error processing the image: {e}")
    return None  # Return None to signal an error in image loading

def predict_image(image_path):
    """Predicts the class of the uploaded image."""
    features = preprocess_image(image_path)
    if features is None:
        return None  # Indicate that preprocessing failed
    prediction = svm.predict(features)[0]
    return prediction

@app.route('/', methods=['GET', 'POST'])
def upload_image():
    if request.method == 'POST':
        if 'file' not in request.files:
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            return redirect(request.url)
        if file:
            filename = file.filename
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(filepath)
            predicted_class = predict_image(filepath)
            if predicted_class is not None:
                if predicted_class == 1:
                    label = 'dog'
                elif predicted_class == 0:
                    label = 'cat'
                else:
                    label = 'invalid input'
                return render_template('result.html', label=label, filename=filename)
            else:
                return render_template('error.html', message="Error: Unable to process the image. Please check the path or file format.")
    return render_template('upload.html')

if __name__ == '__main__':
    if not os.path.exists(app.config['UPLOAD_FOLDER']):
        os.makedirs(app.config['UPLOAD_FOLDER'])
    app.run(debug=True, use_reloader=False)  # Disable the reloader


SVM model loaded from DC.joblib
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [09/Jul/2024 01:44:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2024 01:44:47] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Jul/2024 01:44:47] "GET /static/scripts.js HTTP/1.1" 304 -
127.0.0.1 - - [09/Jul/2024 01:44:47] "GET /static/{{%20url_for('static',%20filename='background.jpg')%20}} HTTP/1.1" 404 -


1/1 [==============================] - 1s 528ms/step


127.0.0.1 - - [09/Jul/2024 01:44:53] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2024 01:44:53] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Jul/2024 01:44:53] "GET /static/uploads/WhatsApp%20Image%202024-05-31%20at%209.42.35%20PM.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2024 01:44:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2024 01:44:58] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Jul/2024 01:44:58] "GET /static/scripts.js HTTP/1.1" 304 -


1/1 [==============================] - 0s 159ms/step


...

127.0.0.1 - - [09/Jul/2024 01:47:29] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2024 01:47:29] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Jul/2024 01:47:29] "GET /static/uploads/68.jpg HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2024 01:47:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2024 01:47:32] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Jul/2024 01:47:32] "GET /static/scripts.js HTTP/1.1" 304 -


1/1 [==============================] - 0s 160ms/step


127.0.0.1 - - [09/Jul/2024 01:47:58] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2024 01:47:58] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Jul/2024 01:47:58] "GET /static/uploads/149.jpg HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2024 01:48:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2024 01:48:00] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Jul/2024 01:48:00] "GET /static/scripts.js HTTP/1.1" 304 -


1/1 [==============================] - 0s 126ms/step


127.0.0.1 - - [09/Jul/2024 01:48:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2024 01:48:17] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Jul/2024 01:48:17] "GET /static/uploads/10.jpg HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2024 01:48:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2024 01:48:19] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [09/Jul/2024 01:48:19] "GET /static/scripts.js HTTP/1.1" 304 -
